In [53]:
import os
import SimpleITK as sitk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import argparse
from NetworkTrainer.network_trainer import *
import torch
from Evaluate.evaluate import *
#from SVVNet.IVDSegmentation.test import copy_sitk_imageinfo
from utils.processing import resize_sitk_image
from utils.tools import csv_to_catalogue

#csv_path = '../../Catalogue' + '/' + str(args.catalogue) + '.csv'
csv_path = 'Catalogue' + '/' + '0.csv'
catalogue = csv_to_catalogue(csv_path)
cases = catalogue['test'].dropna()
list_case_dirs = dict()

cases

0     Case144
1     Case199
2      Case16
3      Case81
4     Case116
5     Case178
6     Case134
7     Case133
8     Case154
9     Case215
10      Case6
11    Case177
12    Case198
13     Case79
14    Case188
15     Case48
16    Case110
17    Case212
18    Case145
19    Case130
20     Case78
21    Case158
Name: test, dtype: object

In [54]:
data_path = '/home/minghao/Desktop/Spine_Segmentation'
save_path = '/home/minghao/Desktop/Output/Joint'

if not os.path.exists(save_path):
    os.mkdir(save_path)

In [55]:
Coccyx_path = '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction'
list_case_dirs['Coccyx'] = [os.path.join(Coccyx_path, cases[i]) for i in range(len(cases))]

In [56]:
list_case_dirs['Coccyx']

['/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case144',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case199',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case16',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case81',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case116',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case178',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case134',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case133',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case154',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case215',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case6',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case177',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case198',
 '/home/minghao/Desktop/Output/Coccyx_Segmentation/Prediction/Case79

In [57]:
IVD_path = '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction'
list_case_dirs['IVD'] = [os.path.join(IVD_path, cases[i]) for i in range(len(cases))]

In [58]:
list_case_dirs['IVD']

['/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case144',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case199',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case16',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case81',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case116',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case178',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case134',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case133',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case154',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case215',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case6',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case177',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case198',
 '/home/minghao/Desktop/Output/IVD_Segmentation/Prediction/Case79',
 '/home/minghao/Desktop/Output/IVD_Segm

In [59]:
Vertebrae_path = '/home/minghao/Desktop/Output/Vertebrae_Segmentation/Prediction'
list_case_dirs['Vertebrae'] = [os.path.join(Vertebrae_path, cases[i]) for i in range(len(cases))]

In [60]:
def copy_sitk_imageinfo(image1, image2):
    image2.SetSpacing(image1.GetSpacing())
    image2.SetDirection(image1.GetDirection())
    image2.SetOrigin(image1.GetOrigin())

    return image2

def landmark_extractor(landmarks):
    """
    return a list of the landmarks
    :param landmarks: pandas.Dataframe
    """
    labels = landmarks.columns[1:].tolist()  # exclude the 'axis' column
    list_landmarks = []
    for label in labels:
        list_landmarks.append(np.array(landmarks[label]))

    return list_landmarks

In [61]:
dict_images = dict()
files_name = ['Coccyx', 'IVD','Vertebrae']
joint = []
S = (12,512,512)

for num in  range(len(cases)):
    Joint_nii = np.zeros(S)

    pre_Coccyx = sitk.ReadImage(list_case_dirs[files_name[0]][num] + '/' + 'pred_' + files_name[0] + 'Mask.nii.gz', sitk.sitkFloat32)
    pre_Coccyx = sitk.GetArrayFromImage(pre_Coccyx)

    Joint_nii = pre_Coccyx

    pre_IVD = sitk.ReadImage(list_case_dirs[files_name[1]][num] + '/' + 'pred_' + files_name[1] + 'Mask.nii.gz', sitk.sitkFloat32)
    pre_IVD = sitk.GetArrayFromImage(pre_IVD)

    pre_Vertebrae = sitk.ReadImage(list_case_dirs[files_name[2]][num] + '/' + 'pred_' + files_name[2] + 'Mask.nii.gz', sitk.sitkFloat32)
    pre_Vertebrae = sitk.GetArrayFromImage(pre_Vertebrae)

    landmark = pd.read_csv(data_path + '/'+ cases[num] + '/landmarks_512.csv')
    landmarks = landmark_extractor(landmark)

    '''for index in range(1,18):
        if np.isnan(landmarks[index][0]):
            pass
        else:
            if index in range(1,9):
                temp = np.where(pre_Vertebrae == index + 1, index + 1, 0)
            if index in range(10,18):
                temp = np.where(pre_IVD == index + 1, index + 1, 0)


            Joint_nii += temp
            #Joint_nii = np.where(Joint_nii > index + 1, index + 1, Joint_nii)'''

    Joint_nii = pre_Coccyx + pre_IVD + pre_Vertebrae
    template_nii = sitk.ReadImage(data_path + '/' + cases[num] + '/MR_512.nii.gz')
    Joint_nii = sitk.GetImageFromArray(Joint_nii)
    Joint_nii = copy_sitk_imageinfo(template_nii, Joint_nii)
    if not os.path.exists(save_path + '/' + cases[num]):
            os.mkdir(save_path + '/' + cases[num])
    sitk.WriteImage(Joint_nii, save_path + '/' + cases[num] + '/pred_JointMask.nii.gz')
print('Done')

Done


In [62]:
Joint_path = '/home/minghao/Desktop/Output/Joint'

list_case_dirs['Joint'] = [os.path.join(Joint_path, cases[i]) for i in range(len(cases))]
dscs = []
for case_dir in range(len(list_case_dirs['Joint'])):

    gt_mask = sitk.ReadImage(data_path + '/' + cases[case_dir] + '/raw_Mask.nii.gz')
    gt = sitk.GetArrayFromImage(gt_mask)
    D, H, W = gt.shape

    pred_mask = sitk.ReadImage(list_case_dirs['Joint'][case_dir] + '/' + 'pred_JointMask.nii.gz')
    pred = resize_sitk_image(pred_mask, dsize=(W, H, None), interpolator='nearest')
    pred = sitk.GetArrayFromImage(pred)

    dsc = cal_subject_level_dice(pred, gt, num_classes=20)
    dscs.append(dsc)

In [63]:
np.mean(dscs)

0.8984552